In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import flopy
import pyemu
import swatmf
import matplotlib.pyplot as plt
swatmf.__version__

In [ ]:
from swatmf import swatmf_pst_utils, swatmf_pst_par

# 1. Set up 
## 1.1 write swatmf.con file

In [ ]:
# working directory and file names
wd = "C:\\Users\\ykishawi2\\Desktop\\18-SWATMODFLOW_UML\\UMLCoupledNov15\\SWAT-MODFLOW"
swat_wd = "C:\\Users\\ykishawi2\\Desktop\\18-SWATMODFLOW_UML\\UMLCoupledNov15\\SWAT Model Folder"

# calibration period
sim_start = '1/1/1995'
warmup = 5
cal_start = '1/1/2000'
cal_end = '12/31/2010'

# time step
time_step = 'month'

# extract simulation (what our targets)
# locations
subs = [37]
grids = [1868, 5092, 7429, 7783, 9134, 15067, 20434, 21187, 22646, 22914, 23607, 24351,
         25198, 25561, 26256, 26388, 27772, 28850, 31134, 32175, 32188, 35890, 43189]

# pilot points included
k_pp = ['hk{}pp.dat'.format(i) for i in range(3)]
sy_pp = ['sy{}pp.dat'.format(i) for i in range(3)]
ss_pp = ['ss{}pp.dat'.format(i) for i in range(3)]
pp_included= k_pp + sy_pp + ss_pp

os.chdir(wd)

In [ ]:
print(pp_included)

In [ ]:
con = swatmf_pst_utils.create_swatmf_con(
    wd, sim_start, warmup, cal_start, cal_end,
    subs=subs, grids=grids, 
    time_step=time_step,
    riv_parm=True,
    pp_included=pp_included
    )

In [ ]:
con

## 1.2 Initiate PEST

In [ ]:
# copy all necessary files (exes) to your working direcotry
swatmf_pst_utils.init_setup(wd, swat_wd)

# 2. Build template files

## 2.1 MODFLOW parameters with Pilot Points

In [ ]:
# m = flopy.modflow.Modflow.load(fs.MODEL_NAM,model_ws=wd,load_only=[]) #<-- load only prevents reading ibound
mname = 'modflow.mfn'
m = flopy.modflow.Modflow.load(mname,
            model_ws=wd
            )
m.check()

In [ ]:
m.bas6.ibound[0].plot()

### Create pilot points as a shapefile
- 25 points with first layer for hk, sy
- 9 points with second
- 3 points with 3rd
- homogeneous with 4th

In [ ]:
#START OF EXAMPLE CELLS
#THIS IS EXAMPLE AND LATER INCLUDED IN THE LOOP, NO NEED TO RUN THIS CELL
# Create pilot points as a shapefile
# we want hk pilot points in the top layer...
prefix_dict = {0:["sy0"]}
df_pp_hk = pyemu.pp_utils.setup_pilotpoints_grid(ml=m,
                                              prefix_dict=prefix_dict,
                                              pp_dir=wd,
                                              tpl_dir=wd,
                                              every_n_cell=30,
                                              shapename='pp_sy0.shp')
# pp_file = os.path.join(working_dir,"sypp.dat")
# assert os.path.exists(pp_file)

So cool, we now defined pilot points as a set of spatially distributed parameters...but how do go from pilot points to the model input HK array? Answer: geostatistics.  We need to calculate the geostatistical factors (weights) used to form the interpolated value for the HK value at each model cell - its a spatially-weighted combination of pilot point values

## Need to create Kriging factors and regularization inputs
Following the guidelines in _Approaches to Highly Parameterized Inversion: Pilot-Point Theory, Guidelines, and Research Directions_ https://pubs.usgs.gov/sir/2010/5168/

### First we need to define a couple geostatistical structures (e.g. variograms)

From _PEST Groundwater Data Utilities Part A: Overview_ page 43, there are 4 acceptable variogram types:

 1. *Spherical*  
### $\gamma\left(h\right)=c\times\left[1.5\frac{h}{a}-0.5\frac{h}{a}^3\right]$ if $h<a$
### $\gamma\left(h\right)=c$ if $h \ge a$  
     
 2. *Exponential*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h}{a}\right)\right]$  
     
 3. *Gaussian*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h^2}{a^2}\right)\right]$  
 
 4. *Power*  
### $\gamma\left(h\right)=c\times h^a$
     
 The number refers to `VARTYPE`. `BEARING` and `ANISOTROPY` only apply if there is a principal direction of anisotropy. $h$ is the separation distance, and $a$ is the range, expressed with the `A` parameter.


### First, let's create ``variogram`` and ``GeoStruct`` objects.  

These describe how HK varies spatailly, remember?

In [ ]:
v = pyemu.geostats.ExpVario(contribution=200,a=30000)
gs = pyemu.geostats.GeoStruct(variograms=v,nugget=0.0)
ax = gs.plot()
ax.grid()
# ax.set_ylim(0,2.0)

In [ ]:
ok = pyemu.geostats.OrdinaryKrige(gs,df_pp_hk)

In [ ]:
#NO NEED TO RUN THIS CELL, THIS IS JUST EXAMPLE, IT IS INCLUDED LATER IN THE LOOP
df = ok.calc_factors_grid(
            m.sr,
            var_filename= "sy0pp.var.ref",
            minpts_interp=1,
            maxpts_interp=30,
            search_radius=200000,
            verbose=True,
            num_threads=12)

One of the really cool things about geostatistics is that it gives you both the interpolation (factors), but also gives you the uncertainty in the areas between control (pilot) points.  Above, we wrote this uncertainty information to an array that has the same rows and cols as the model grid - this array is very useful for understanding the function of the variogram.

In [ ]:
#NO NEED TO RUN THIS CELL, THIS IS JUST EXAMPLE, IT IS INCLUDED LATER IN THE LOOP

# arr_var = np.loadtxt(pst_name.replace(".pst",".var.ref"))
arr_var = np.loadtxt("sy0pp.var.ref")
ax = plt.subplot(111,aspect="equal")
p = ax.imshow(arr_var,extent=m.sr.get_extent(),alpha=0.25)
plt.colorbar(p)
ax.scatter(df_pp_hk.x,df_pp_hk.y,marker='.',s=4,color='r')
#END OF EXAMPLE CELLS

##

In [ ]:
#THE PREVIOUS CELLS ARE EXAMPLE, THIS CELL IS THE CONTINUATION OF SETUP CODE
lyrs = 3
cell_nums = [30, 50, 80]
hk_prefix = ['hk{}'.format(i) for i in range(lyrs)]
sy_prefix = ['sy{}'.format(i) for i in range(lyrs)]
ss_prefix = ['ss{}'.format(i) for i in range(lyrs)]


In [ ]:
sy_prefix

In [ ]:
#LOOP CELL
#THIS CELL SHOULD BE REPEATED AS THE NUMBER OF MODFLOW PARAMTER WE WANT TO PREPARE
#HERE IS HAS sy_prefix, then we choose the correct contribtion and correct a, then hk and change contribution and a and so on
##ss_prefix contribution=2.4e-6;   sy_prefix contribution=0.8;   hk_prefix contribution=200
for s, c in zip(hk_prefix, cell_nums):
    prefix_dict = {0:[s]}
    df_pp = pyemu.pp_utils.setup_pilotpoints_grid(ml=m,
                                                  prefix_dict=prefix_dict,
                                                  pp_dir=wd,
                                                  tpl_dir=wd,
                                                  every_n_cell=c,
                                                  shapename='pp_{}.shp'.format(s))
    v = pyemu.geostats.ExpVario(contribution=200,a=30000)
    gs = pyemu.geostats.GeoStruct(variograms=v,nugget=0.0)
    ok = pyemu.geostats.OrdinaryKrige(gs,df_pp)
    df = ok.calc_factors_grid(
                m.sr,
                var_filename= "{}pp.var.ref".format(s),
                minpts_interp=1,
                maxpts_interp=30,
                search_radius=200000,
                verbose=True,
                num_threads=12)    
    ok.to_grid_factors_file("{}pp.dat.fac".format(s))

## 2.2 River parameters with Pilot Points

In [ ]:
# provide channel ids that will be used for calibration
rivgs = ['rg002', 'rg003', 'rg004', 'rg009', 'rg010']
swatmf_pst_par.create_riv_par(wd, rivgs)

In [ ]:
# create a template file for mf_riv.par file
swatmf_pst_utils.riv_par_to_template_file('mf_riv.par')

In [ ]:
# overwrite the river package file
swatmf_pst_par.riv_par(wd)

## 2.3 SWAT model.in file

In [ ]:
# model.in file used
sw_par = swatmf_pst_utils.model_in_to_template_file()
sw_par

# 3. Create instruction files

## 3.1 Depth to watertable (MODFLOW) 

In [ ]:
grids

In [ ]:
swatmf_pst_utils.extract_depth_to_water(grids, sim_start, cal_end)
#There is a problem creating the last three grid wells 32188, 35890, 43189
# I also removed these three wells from modflow.obd

In [ ]:
# get dtw col nams 
dtw_df = pd.read_csv(
                    "modflow.obd",
                    sep='\t',
                    index_col=0,
                    header=0,
                    parse_dates=True,
                    na_values=[-999, ""])
dtw_df

In [ ]:
sim_files = ['dtw_{}.txt'.format(i) for i in grids]
obd_cols = ['W{}'.format(i) for i in grids]

In [ ]:
sim_files

In [ ]:
obd_cols

In [ ]:
for s, o in zip(sim_files, obd_cols):
    swatmf_pst_utils.mf_obd_to_ins(s, o, cal_start, cal_end)

## 4.1 Streamflow (SWAT)

In [ ]:
# extract daily stream discharge
swatmf_pst_utils.extract_month_stf(subs, sim_start, warmup, cal_start, cal_end)

In [ ]:
# create stf_mon.obd
stf_obd = pd.read_csv(
                    'streamflow.obd',
                    sep='\t',
                    usecols=['Date', 'sub_37'],
                    index_col=0,
                    parse_dates=True,
                    na_values=[-999, '']
                    )
stf_obd = stf_obd.resample('M').mean()
stf_obd.to_csv('stf_mon.obd', sep='\t', float_format='%.7e')

## 4.2 match it with dtw_obd file (MODFLOW)

In [ ]:
swatmf_pst_utils.stf_obd_to_ins?

In [ ]:
swatmf_pst_utils.stf_obd_to_ins('str_037.txt', 'sub_37',cal_start, cal_end, time_step='month')

# 5. Create PEST control file

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'uml_dummy.pst')
io_files

In [ ]:
par = pst.parameter_data
par

## 5.1 Assign parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    elif par.iloc[i, 0][:2] == 'ss':
        par.iloc[i, 6] = 'ss'
    else:
        par.iloc[i, 6] = 'swat'
print(par)

## 5.2 Adjust initial parameter values and their ranges

In [ ]:
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk'):
        par.iloc[i, 3] = 18               #initial value
        par.iloc[i, 4] = 4.200000e-01    #lower value
        par.iloc[i, 5] = 4.050000e+02    #upper value
    elif (par.iloc[i, 6] == 'sy'):
        par.iloc[i, 3] = 2.000000e-01 #intial value       
        par.iloc[i, 4] = 1.000000e-04 #lower value
        par.iloc[i, 5] = 0.6          #upper value
    elif (par.iloc[i, 6] == 'ss'):
        par.iloc[i, 3] = 3.300000e-06 #inital value       
        par.iloc[i, 4] = 3.000000e-06 #lower value
        par.iloc[i, 5] = 3.400000e-06 #upper value
    elif (par.iloc[i, 6] == 'rivbot'):
        par.iloc[i, 3] = 3.001     
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 6
        par.iloc[i, 8] = -3
    elif (par.iloc[i, 6] == 'rivcd'):
        par.iloc[i, 3] = 50.001       
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -50
    else:
        count += 1
count

In [ ]:
# CN2
par.loc['cn2', 'parval1'] = 66
par.loc['cn2', 'parlbnd'] = 35
par.loc['cn2', 'parubnd'] = 92
par.loc['cn2', 'offset'] = -1

# sol_k()
par.loc['sol_k()', 'parval1'] = 12
par.loc['sol_k()', 'parlbnd'] = 0.7
par.loc['sol_k()', 'parubnd'] = 57
par.loc['sol_k()', 'offset'] = -1

# sol_awc()
par.loc['sol_awc()', 'parval1'] = 1.001
par.loc['sol_awc()', 'parlbnd'] = 0.5
par.loc['sol_awc()', 'parubnd'] = 1.5
par.loc['sol_awc()', 'offset'] = -1

# ESCO
par.loc['esco', 'parval1'] = 1.001
par.loc['esco', 'parlbnd'] = 0.5
par.loc['esco', 'parubnd'] = 1.5
par.loc['esco', 'offset'] = -1


In [ ]:
par

## 5.3 Assign parameter group name

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    if obd.iloc[i, 0][:3] == 'sub':
        obd.iloc[i, 3] = obd.iloc[i, 0][:-7]
    else:
        obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

## 5.4 Provide actual observed values to control file

In [ ]:
# Streamflow
stf_obd = pd.read_csv('stf_mon.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

In [ ]:
# watertable
dtw_obd = pd.read_csv('modflow.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
dtw_obd = dtw_obd[cal_start: cal_end]
dtw_obd

In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
stf_obd

In [ ]:
dtw_obd

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += dtw_obd[i].dropna().tolist()
    print(i)

len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 6. Create new control file with settings

In [ ]:
pst.control_data.noptmax=0
pst.model_command = 'python forward_run.py'
pst.write('uml_pest.pst')